In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')
sns.set_context('notebook')

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix,classification_report, roc_curve, log_loss, brier_score_loss, roc_auc_score, make_scorer
from timeit import default_timer as timer

In [2]:
# Custom loss function - Binned Sum of Squared Residuals

def binned_sum_of_squared_residuals(y_actual, y_predicted):

    df = pd.DataFrame(y_actual.copy())
    df.columns = ['Actual']
    df['Predicted Probability'] = y_predicted

    bins = np.arange(0,1.05,0.05)
    labels = ['0.05','0.1','0.15','0.2','0.25','0.3','0.35','0.4','0.45','0.5',
              '0.55','0.6','0.65','0.7','0.75','0.8','0.85','0.9','0.95','1']
    df['Bin'] = pd.cut(df['Predicted Probability'], bins = bins, labels = labels)
    bin_df = df.groupby('Bin')['Actual', 'Predicted Probability'].mean()
    bin_df.reset_index(inplace = True)
    bin_df.columns = ['Bin', 'Actual', 'Predicted']

    binned_sum_of_squares = sum(((bin_df.Actual - bin_df.Predicted).fillna(0)*100) ** 2)
    
    return binned_sum_of_squares

# wrapper for grid search object
bssr = make_scorer(binned_sum_of_squared_residuals, 
                   greater_is_better = False,
                   needs_proba = True)

In [3]:
# Training Function

def train(model, X_train, y_train):
    
    start_training = timer()

    model.fit(X_train, np.ravel(y_train))

    end_training = timer()
    print('Training Time:', round(end_training - start_training,1), 'seconds')

In [5]:
# Prediction Function

def predict(model, X_validation):
    
    start_prediction = timer()

    y_pred = pd.DataFrame(model.predict(X_validation), index = X_validation.index)
    y_pred_prob = pd.DataFrame(model.predict_proba(X_validation)[:,1], index = X_validation.index)

    end_prediction = timer()
    print('Prediction Time:', round(end_prediction - start_prediction,1), 'seconds')
    
    return y_pred, y_pred_prob

In [6]:
# Reporting Function

def reports(model, X_columns, y_validation, y_pred, y_pred_prob):

    start_reports = timer()

    print('Confusion Matrix:\n', confusion_matrix(y_validation, y_pred))
    print('Classification Report:\n', classification_report(y_validation, y_pred))
    print('Accuracy:', round((sum(y_pred.iloc[:,0] == y_validation.iloc[:,0]) / len(y_pred.values) * 100),2), "%")
    print('Log Loss:', log_loss(y_validation, y_pred_prob))
    print('Brier Score Loss:', brier_score_loss(y_validation, y_pred_prob))

    fpr, tpr, thresholds = roc_curve(y_validation, y_pred_prob)

    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.show()

    print('ROC AUC Score:', roc_auc_score(y_validation, y_pred_prob))
    
    if isinstance(model, RandomForestClassifier):
    
        forest_feature_importance = pd.DataFrame(zip(X_columns,model.feature_importances_))
        forest_feature_importance.sort_values(1, inplace = True)
        forest_feature_importance.reset_index(inplace = True, drop = True)

        plt.bar(forest_feature_importance[0][-20:],forest_feature_importance[1][-20:])
        plt.xticks(rotation = 'vertical')
        plt.title('20 Largest Importances')
        plt.show()

        plt.bar(forest_feature_importance[0][0:20],forest_feature_importance[1][0:20])
        plt.xticks(rotation = 'vertical')
        plt.title('20 Smallest Importances')
        plt.show()
        
    else:
        
        feature_coef = pd.DataFrame(zip(X_columns,model.coef_[0]))
        feature_coef.sort_values(1, inplace = True)
        feature_coef.reset_index(inplace = True, drop = True)

        plt.bar(feature_coef[0][-20:],feature_coef[1][-20:])
        plt.xticks(rotation = 'vertical')
        plt.title('20 Largest Feature Coefficients')
        plt.show()

        plt.bar(feature_coef[0][0:20],feature_coef[1][0:20])
        plt.xticks(rotation = 'vertical')
        plt.title('20 Smallest Feature Coefficients')
        plt.show()

    proba_df = y_validation.copy()
    proba_df['Predicted Probability'] = y_pred_prob

    #The below bins by Predicted Probability
    bins = np.arange(0,1.05,0.05)
    labels = ['0.05','0.1','0.15','0.2','0.25','0.3','0.35','0.4','0.45','0.5',
              '0.55','0.6','0.65','0.7','0.75','0.8','0.85','0.9','0.95','1']
    proba_df['Bin'] = pd.cut(proba_df['Predicted Probability'], bins = bins, labels = labels)
    #proba_df.reset_index(drop = True, inplace = True)
    chart_df = proba_df.groupby('Bin')['Solved', 'Predicted Probability'].mean()
    chart_df['Count'] = proba_df.groupby('Bin')['Solved'].count()
    chart_df.reset_index(inplace = True)
    chart_df.columns = ['Bin', 'Actual', 'Predicted', 'Count']

    print(chart_df)

    plt.bar(chart_df.Bin, chart_df.Count)
    plt.xticks(rotation = 45)
    plt.title('Frequency of Test Cases by Predicted Clearance Probability')
    plt.xlabel('Case Clearance Probability')
    plt.ylabel('Number of Cases')
    plt.show()

    plt.scatter(chart_df.Bin, chart_df.Actual, label = 'Actual', alpha = 1)
    plt.scatter(chart_df.Bin, chart_df.Predicted, color = 'r', marker = 'D', label = 'Predicted', alpha = 0.75)
    plt.legend()
    plt.title('Actual and Predicted Clearance Rate by 5% Bin')
    plt.xticks(rotation = 45)
    plt.xlabel('Bin')
    plt.yticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
    plt.ylabel('Clearance Rate')
    plt.show()

    plt.scatter(chart_df.Predicted, chart_df.Actual)
    plt.plot([0,1],[0,1], linestyle = '--', color = 'g')
    plt.title('Actual vs. Predicted Clearance Rate by 5% bin')
    plt.xticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],rotation = 45)
    plt.xlabel('Predicted')
    plt.yticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
    plt.ylabel('Actual')
    plt.show()

    plt.bar(chart_df.Bin, chart_df.Actual - chart_df.Predicted,)
    plt.title('Actual - Predicted Clearance Rate by 5% Bin')
    plt.xticks(rotation = 45)
    plt.xlabel('Bin (Range = Value - 5% to Value)')
    #plt.yticks([-0.05,-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04,0.05])
    plt.ylabel('Delta Clearance Rate')
    plt.show()

    print('Sum of Squared Binned Residuals:', binned_sum_of_squared_residuals(y_validation, y_pred_prob))

    end_reports = timer()
    print('Reporting Time:', round(end_reports - start_reports,1), 'seconds')
    
    return proba_df, chart_df

In [7]:
# Load data into DataFrame
murders_df = pd.read_csv('C:\\Users\\Work_Remote\\Desktop\\murders_cleaned.csv')
murders_df.columns

Index(['Agentype', 'Solved', 'Year', 'Month', 'Homicide', 'Situation',
       'VicAge', 'VicSex', 'VicRace', 'VicEthnic', 'Weapon', 'VicCount',
       'AgencyCases', 'WhiteVictimPercent', 'AgeKnown'],
      dtype='object')

In [23]:
murders_df.Year

0         1976
1         1976
2         1976
3         1976
4         1976
          ... 
804713    2015
804714    2017
804715    2017
804716    2018
804717    2019
Name: Year, Length: 804718, dtype: int64

In [32]:
start_preprocessing = timer()

# Encode y column
murders_df.Solved[murders_df.Solved == 'Yes'] = 1
murders_df.Solved[murders_df.Solved == 'No'] = 0

# Divide categorical and numerical features
X_categorical = murders_df[[
                        'AgeKnown',
                        'Agentype',
                        'Month',
                        'Homicide',
                        'Situation',
                        'VicSex',
                        'VicRace',
                        'VicEthnic',
                        'Weapon'
                            ]].reset_index(drop = True)

X_numerical = murders_df[[
                        'VicAge',
                        'VicCount',
                        'AgencyCases',
                        'WhiteVictimPercent',
                        'Year'
                        ]].reset_index(drop = True)

#Create y series
y = pd.DataFrame(murders_df.Solved).reset_index(drop = True)

#Create dummy columns for categorical features
X = pd.get_dummies(X_categorical, drop_first = True)

#Add numerical features to model dataframe
X = pd.concat([X, X_numerical], axis = 1)

#Scale features
X = pd.DataFrame(StandardScaler().fit_transform(X), index = X.index, columns = X.columns)

#Create training set from all years except most recent
X_train = X[murders_df.Year != 2019]
y_train = y[murders_df.Year != 2019].astype('int')
#Create most recent year test set
X_test = X[murders_df.Year == 2019]
y_test = y[murders_df.Year == 2019].astype('int')

end_preprocessing = timer()
print('Preprocessing Time:', round(end_preprocessing - start_preprocessing,1), 'seconds')

<ipython-input-32-541794f82b9f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  murders_df.Solved[murders_df.Solved == 'Yes'] = 1
<ipython-input-32-541794f82b9f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  murders_df.Solved[murders_df.Solved == 'No'] = 0


Preprocessing Time: 3.6 seconds


In [13]:
y_train.value_counts()

Solved
1         426019
0         177519
dtype: int64

In [14]:
# Create balanced training set

solved_count, unsolved_count = y_train.value_counts()
solved = y_train[y_train.Solved == 1]
unsolved = y_train[y_train.Solved == 0]
solved_sample = solved.sample(unsolved_count, random_state = 33)

y_train_balanced = pd.concat([unsolved, solved_sample], axis = 0)

X_train_balanced = y_train_balanced.join(X_train,how = 'inner').drop('Solved', axis =1)

In [ ]:
model  = RandomForestClassifier(n_jobs = 9, 
                            random_state = 33,
                            max_depth = 25,
                            n_estimators = 1250,
                            min_samples_leaf = 1)

# Balanced? Wait for results

train(model, X_train, y_train)
y_pred, y_pred_prob = predict(model, X_test)
proba_df, chart_df = reports(model, X_train.columns, y_test, y_pred, y_pred_prob)